# Taza de Suicidios
#### Jose Carlos Pimienta Ibarra

En este proyecto se evaluara informacion sobre informacion de suicidios por pais desde 1985 a 2016. Esta informacion el cual fue encontrado en kaggle fue recopilado desde distintos sitios, los cuales incluyen el banco mundial, la organizacion mundial de la salud entre otros.

El dataset incluye las sigueinte columnas:
- País
- Año
- Sexo
- Edad
- Número de suicidios
- Población
- Suicidos por cada 100,000 habitantes
- Pais-Año
- Índice de desarrollo humano por año
- Producto Interno Bruto por año (PIB)
- PIB per capita
- Generación
    

Importamos las librerias de pyspark y lo inicializamos el contexto para poder usarlo

In [3]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import sql
sc = SparkContext.getOrCreate()
sqlContext = sql.SQLContext(sc)

In [11]:
data = (sqlContext.read.format("com.databricks.spark.csv")
        .option("header", "true")
        .load("master.csv"))
print(data)

DataFrame[country: string, year: string, sex: string, age: string, suicides_no: string, population: string, suicides/100k pop: string, country-year: string, HDI for year: string,  gdp_for_year ($) : string, gdp_per_capita ($): string, generation: string]


En la impresion anterior podemos ver el tipo de valor que tiene cada columna el dataframe.

A continuación se mostraran los primeros 20 renglones de dataframe generado

In [12]:
data.show()

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+------------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|        null|     2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|        null|     2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4

 Podemos ver que en los primeros 20 renglones el valor de la columna **HDI for year** es **null**.
 Usando la herramiente _openRefine_ podemos ver que en esa columna de los __27,820__ registros **19456** son nulos, esto significa que en la columna **HDI for year** el ** _69.9%_ ** de los datos  es nulo por lo que podemos decir que esa columnna no nos servira para analisis por lo que podemos eliminar esa colummna.

In [16]:
print(data.columns)

['country', 'year', 'sex', 'age', 'suicides_no', 'population', 'suicides/100k pop', 'country-year', 'HDI for year', ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation']
